In [1]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
%pylab inline
plt.style.use('seaborn-dark')
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import matplotlib as plt
plt.rc('font', size=14)

def plot_2d_separator(classifier, X, fill=False, line=True, ax=None, eps=None):
    if eps is None:
        eps = 1.0
    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 100)
    yy = np.linspace(y_min, y_max, 100)
    
    X1, X2 = np.meshgrid(xx, yy)
    X_grid = np.c_[X1.ravel(), X2.ravel()]
    try:
        decision_values = classifier.decision_function(X_grid)
        levels = [0]
        fill_levels = [decision_values.min(), 0, decision_values.max()]
    except:
        decision_values = classifier.predict_proba(X_grid)[:, 1]
        levels = [.5]
        fill_levels = [0, .5, 1]
        
    if ax is None:
        ax = plt.gca()
    if fill:
        ax.contourf(X1, X2, decision_values.reshape(X1.shape),
                    levels=fill_levels, colors=['cyan', 'pink'])
    if line:
        ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=levels,
                   colors="black")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())

Populating the interactive namespace from numpy and matplotlib


## Загружаем данные

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
print(train.shape, test.shape)

(112500, 11) (37500, 10)


In [4]:
y = train.pop('плохой_клиент')
train.shape, y.shape

((112500, 10), (112500,))

## Предобработка

## Очень много пропусков в доходе: 

In [5]:
print(train.columns)
print(train.info())

Index(['линии', 'возраст', 'поведение_30-59_дней', 'Debt_Ratio', 'доход',
       'число_кредитов', 'поведение_90_дней', 'недвижимость',
       'поведение_60-89_дней', 'семья'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112500 entries, 0 to 112499
Data columns (total 10 columns):
линии                   112500 non-null float64
возраст                 112500 non-null int64
поведение_30-59_дней    112500 non-null int64
Debt_Ratio              112500 non-null float64
доход                   90274 non-null float64
число_кредитов          112500 non-null int64
поведение_90_дней       112500 non-null int64
недвижимость            112500 non-null int64
поведение_60-89_дней    112500 non-null int64
семья                   109549 non-null float64
dtypes: float64(4), int64(6)
memory usage: 8.6 MB
None


In [6]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37500 entries, 0 to 37499
Data columns (total 10 columns):
линии                   37500 non-null float64
возраст                 37500 non-null int64
поведение_30-59_дней    37500 non-null int64
Debt_Ratio              37500 non-null float64
доход                   29995 non-null float64
число_кредитов          37500 non-null int64
поведение_90_дней       37500 non-null int64
недвижимость            37500 non-null int64
поведение_60-89_дней    37500 non-null int64
семья                   36527 non-null float64
dtypes: float64(4), int64(6)
memory usage: 2.9 MB
None


## Интересный человек №21595

In [7]:
train.sort_values(by='недвижимость', ascending=False).head()

,линии,возраст,поведение_30-59_дней,Debt_Ratio,доход,число_кредитов,поведение_90_дней,недвижимость,поведение_60-89_дней,семья
21595,0.003032,53,0,5.967504,8000.0,58,0,54,0,0.0
91547,0.010846,60,0,1.041757,70000.0,38,0,32,0,0.0
44022,0.036942,43,0,0.719174,55966.0,41,0,29,0,0.0
57006,0.985390,51,1,0.515205,56000.0,42,0,25,0,1.0
71133,0.002280,73,0,3.593683,14500.0,26,0,23,0,0.0


In [8]:
train.describe()

,линии,возраст,поведение_30-59_дней,Debt_Ratio,доход,число_кредитов,поведение_90_дней,недвижимость,поведение_60-89_дней,семья
count,112500.000000,112500.000000,112500.000000,112500.000000,9.027400e+04,112500.000000,112500.000000,112500.000000,112500.000000,109549.000000
mean,6.057904,52.292702,0.425538,350.965444,6.662380e+03,8.464267,0.271627,1.019680,0.245333,0.755662
std,257.959993,14.765029,4.246085,1864.557746,1.540162e+04,5.149137,4.223498,1.130324,4.208686,1.113419
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.029840,41.000000,0.000000,0.175069,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.154015,52.000000,0.000000,0.366336,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.559389,63.000000,0.000000,0.863569,8.250000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,50708.000000,109.000000,98.000000,326442.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,13.000000


In [9]:
train['доход'][train["доход"] < 1].value_counts()

0.0    1215
Name: доход, dtype: int64

## Заполним пропуски в доходе средним значением по всей таблице, а значение семьи = 0 

In [10]:
train.sort_values(by='доход', ascending=False).head(10)

,линии,возраст,поведение_30-59_дней,Debt_Ratio,доход,число_кредитов,поведение_90_дней,недвижимость,поведение_60-89_дней,семья
21377,0.007328,52,0,0.001470,3008750.0,10,0,1,0,3.0
57127,0.000000,68,0,0.002776,1794060.0,15,0,1,0,0.0
1124,0.163856,44,0,0.004013,1560100.0,12,0,4,0,2.0
65022,0.469025,44,1,0.004537,1072500.0,9,0,2,0,1.0
61811,0.071628,50,0,0.008268,702500.0,12,0,4,0,2.0
78489,0.064155,52,0,0.003698,699530.0,11,0,2,0,1.0
64092,0.000000,83,0,0.000000,629000.0,3,0,0,0,0.0
40095,0.079804,61,0,0.001547,562466.0,7,0,0,0,1.0
58209,0.946867,44,2,0.002459,440000.0,2,0,0,1,0.0
62414,0.036010,43,0,0.002557,428250.0,13,0,1,0,1.0


In [11]:
mean_income_train = train['доход'][train['доход'] < 3800000].mean()
mean_income_test = test['доход'][test['доход'] < 3800000].mean()
print(mean_income_train, mean_income_test)
train['доход'].fillna(mean_income_train, inplace=True)
test['доход'].fillna(mean_income_test, inplace=True)

6662.379910051621 6693.8207701283545


### Семья

In [13]:
train['семья'].fillna(0, inplace=True)
test['семья'].fillna(0, inplace=True)

## Так-то лучше.

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112500 entries, 0 to 112499
Data columns (total 10 columns):
линии                   112500 non-null float64
возраст                 112500 non-null int64
поведение_30-59_дней    112500 non-null int64
Debt_Ratio              112500 non-null float64
доход                   112500 non-null float64
число_кредитов          112500 non-null int64
поведение_90_дней       112500 non-null int64
недвижимость            112500 non-null int64
поведение_60-89_дней    112500 non-null int64
семья                   112500 non-null float64
dtypes: float64(4), int64(6)
memory usage: 8.6 MB


## Тренировка моделей

In [15]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, max_depth=2, random_state=0)
model.fit(train, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [16]:
a = model.predict_proba(test)[:, 1]


## Оценка качества с помощью кросс-валидации:

In [18]:
from sklearn.model_selection import StratifiedKFold, train_test_split
train_toTrain, train_postpone, y_toTrain, y_postpone = \
            train_test_split(train.values, y, test_size=0.2, random_state=0)

model_rand_forest = RandomForestClassifier(n_estimators=200, max_depth=2, random_state=0)

model_rand_forest.fit(train_toTrain, y_toTrain)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [19]:
from sklearn.metrics import accuracy_score

model_rand_forest_prediction = model_rand_forest.predict(train_postpone)
accuracy_score(y_postpone, model_rand_forest_prediction)

0.9349333333333333

## Настроим параметры случайного леса:

In [20]:
from sklearn.model_selection import GridSearchCV, cross_val_score

In [21]:
model_rand_forest_params = {'n_estimators' : range(100, 120), 'max_depth' : range(1, 4)}
model_rand_forest_grid = GridSearchCV(model_rand_forest, model_rand_forest_params, cv=5, n_jobs=-1, verbose=True)

In [ ]:
model_rand_forest_grid.fit(train_toTrain, y_toTrain)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   35.9s


In [14]:
model_rand_forest_grid.best_params_


{'max_depth': 3, 'n_estimators': 107}

In [16]:
model_rand_forest_grid.best_score_

0.9339888888888889

In [17]:
accuracy_score(y_postpone, model_rand_forest_grid.predict(train_postpone))

0.9357333333333333

In [22]:
#model_rand_forest_grid.predict()
#pd.DataFrame('id' : np.arange(37500), 'a' : mode

## Попробуем найти оптимальную максимальную глубину деревьев для n_estimators = 107

In [71]:
model_rand_forest_params = {'max_depth' : np.arange(1, 10), 'n_estimators' : np.arange(107, 200)}
model_rand_forest_grid = GridSearchCV(model_rand_forest, model_rand_forest_params, cv=5,
                                      n_jobs=-1, verbose=True)
model_rand_forest_grid.fit(train_toTrain, y_toTrain)

Fitting 5 folds for each of 837 candidates, totalling 4185 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 38.3min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 65.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 103.9min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 158.7min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 230.8min
[Parallel(n_jobs=-1)]: Done 4185 out of 4185 | elapsed: 246.5min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), 'n_estimators': array([107, 108, ..., 198, 199])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [72]:
print(model_rand_forest_grid.best_params_, model_rand_forest_grid.best_score_, sep="\n")
#0.9363555555555556

{'max_depth': 8, 'n_estimators': 126}
0.9365666666666667


In [73]:
accuracy_score(y_postpone, model_rand_forest_grid.predict(train_postpone))
#0.9383111111111111

0.9376444444444444

In [74]:
model_rand_forest_best = RandomForestClassifier(n_estimators=126,
                                               max_depth=8, 
                                               random_state=0)
model_rand_forest_best.fit(train, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=126, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [86]:
model_rand_forest_prediction = model_rand_forest_grid.predict(train)
print(np.mean(model_rand_forest_prediction == y))

0.9411466666666667


In [87]:
a = model_rand_forest_best.predict_proba(test)[:, 1]

In [88]:
model_rand_forest_prediction = model_rand_forest_grid.predict_proba(test)[:, 1]
pd.DataFrame({'id' : np.arange(37500), 'a' : model_rand_forest_prediction}).to_csv('./solution_rand_forest_grid.csv', index=False)

In [101]:
#print(a)
predi = model_rand_forest_grid.predict_proba(test)[:, 1]
predi[predi > 0.8]
test['predict'] = predi
#pd.DataFrame(model_rand_forest_best.predict_proba(test), columns=model_rand_forest_best.classes_)

In [106]:
test[test['predict'] > 0.7].head(15)

,линии,возраст,поведение_30-59_дней,Debt_Ratio,доход,число_кредитов,поведение_90_дней,недвижимость,поведение_60-89_дней,семья,predict
2473,1.000000,28,2,892.000000,6693.82077,5,3,0,2,3.0,0.724707
3150,1.000000,60,98,0.000000,1375.00000,0,98,0,98,0.0,0.708183
4665,1.980066,39,0,2036.000000,6693.82077,6,3,1,2,0.0,0.727528
5008,0.961229,42,4,0.518629,7675.00000,16,1,2,2,2.0,0.712943
5291,1.224775,37,4,1921.000000,6693.82077,9,2,1,3,0.0,0.760816
5320,1.000000,35,98,10.000000,6693.82077,0,98,0,98,0.0,0.845723
5713,1.003322,43,5,5965.000000,6693.82077,13,3,2,1,0.0,0.705086
6169,1.000000,46,98,77.000000,6693.82077,0,98,0,98,0.0,0.798217
7429,1.033630,48,1,0.539093,7916.00000,4,5,1,2,4.0,0.817465
7540,1.000000,31,0,503.000000,6693.82077,4,7,0,7,0.0,0.727123


In [107]:
test.head()

,линии,возраст,поведение_30-59_дней,Debt_Ratio,доход,число_кредитов,поведение_90_дней,недвижимость,поведение_60-89_дней,семья,predict
0,0.853506,67,3,1.146778,2513.00000,7,3,1,0,1.0,0.568034
1,0.596972,53,0,2620.000000,6693.82077,13,0,1,0,0.0,0.064374
2,0.308867,43,0,0.191476,29000.00000,15,0,3,0,4.0,0.028298
3,0.032604,66,0,0.641992,5200.00000,9,0,0,0,0.0,0.011597
4,0.002666,38,0,0.316677,3261.00000,3,0,1,0,0.0,0.018517


### Возможно, признаки "пповедене 30-59_дней, 60-89 дублируют поведение_90_дней. Попробуем их убрать и посмотрим на результат

### Замечены странные значения этих признаков - 98. При этом признак линии всегда имеет значение 0.99... Возможно, нужно избавиться от таких объектов